**Update Feb 8: ** The sentiment analysis works but the original texts are all neutral so it won't have an impact of the predictions. However I will use it for now as a categorical value where "0" = Has captions and "-2" = "no caption".

Doing this analysis made me realize that I have to write captions where emotions are present so my audience feels more conected.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import (CountVectorizer, TfidfVectorizer)
from sklearn.naive_bayes import (GaussianNB, MultinomialNB)
import re 
import nltk 
from sklearn.metrics import (accuracy_score, ConfusionMatrixDisplay)
from nltk.corpus import stopwords
from nltk import ngrams
nltk.download("stopwords")
stop_words = stopwords.words("spanish") # stopwords palabras de relleno

# Análisis de texto clásico ciertas palabras no añaden contexto y en base al resto de palabras podemos entender las relaciones que permitan hacer la clasficación
# Función para quitar los stopwords del data frame
def remove_stopwords(text):
    clean_text = []
    text = text.lower()
    if text is not None:
        words = text.split()
        for word in words:
            if word not in stop_words:
                clean_text.append(word)
        return " ".join(clean_text)
    else:
        return None


def remove_symbols(text):
    if text is not None:
        text = re.sub(r'''[.,?*&'"+${}-]''','',text) # Sub sustituir 
        return text
    else:
        return None

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Open the Dataset
df_raw = pd.read_csv("datasets/extras/archivo_combinado.csv")
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   identificador_de_la_publicacion  702 non-null    int64  
 1   identificador_de_la_cuenta       702 non-null    int64  
 2   nombre_de_usuario_de_la_cuenta   702 non-null    object 
 3   nombre_de_la_cuenta              702 non-null    object 
 4   descripcion                      302 non-null    object 
 5   duracion_segundos                702 non-null    int64  
 6   hora_de_publicacion              702 non-null    object 
 7   enlace_permanente                13 non-null     object 
 8   tipo_de_publicacion              702 non-null    object 
 9   comentario_sobre_los_datos       0 non-null      float64
 10  fecha                            702 non-null    object 
 11  alcance                          642 non-null    float64
 12  me_gusta              

In [6]:
# Create a Dataframe only with the id and description
selected_columns = ["identificador_de_la_publicacion", "descripcion"]
df_interim = df_raw[selected_columns]
df_interim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   identificador_de_la_publicacion  702 non-null    int64 
 1   descripcion                      302 non-null    object
dtypes: int64(1), object(1)
memory usage: 11.1+ KB


In [7]:
# Remove rows with NaN
df_interim = df_interim.dropna(subset=["descripcion"])
df_interim.info()


<class 'pandas.core.frame.DataFrame'>
Index: 302 entries, 0 to 701
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   identificador_de_la_publicacion  302 non-null    int64 
 1   descripcion                      302 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.1+ KB


In [8]:
# Clean data
df_interim["message_clean"] = df_interim["descripcion"].apply(lambda x: remove_stopwords(x))
df_interim["message_clean"] = df_interim["message_clean"].apply(lambda x: remove_symbols(x))
df_interim["message_clean"] = df_interim["message_clean"].str.replace(":", "")
df_interim["message_clean"] = df_interim["message_clean"].str.replace("!", "")
df_interim["message_clean"] = df_interim["message_clean"].str.replace("¡", "")
df_interim["message_clean"] = df_interim["message_clean"].str.replace("(", "")
df_interim["message_clean"] = df_interim["message_clean"].str.replace(")", "")
df_interim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 302 entries, 0 to 701
Data columns (total 3 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   identificador_de_la_publicacion  302 non-null    int64 
 1   descripcion                      302 non-null    object
 2   message_clean                    302 non-null    object
dtypes: int64(1), object(2)
memory usage: 9.4+ KB


In [9]:
df_interim.sample(10)

,identificador_de_la_publicacion,descripcion,message_clean
164,17878006919851859,Tengo ganas de estar \nmás cerca de ti,ganas cerca
177,18253901530159555,Sueños despiertos,sueños despiertos
614,18455532748038607,¡No podía faltar \nMi platillo \nfavorito!,no podía faltar platillo favorito
660,18072687724546137,Buen inicio \nde semana,buen inicio semana
265,18003645379797478,Logros en Youtube,logros youtube
256,17974825862419588,En camino a cumplir un sueño,camino cumplir sueño
641,18313632556093128,¡Hoy voy a probar \nalgo nuevo! ¡Te cuento \np...,hoy voy probar nuevo te cuento pronto cómo fue...
152,18248626282087562,Te extraño mucho amiga,extraño amiga
379,17933438987683652,¿Asistirías a un webinar GRATUITO sobre Sueños...,¿asistirías webinar gratuito sueños premonitorios
673,18050136700849085,"Hoy agradezco ser mamá, aún con todo lo difíci...",hoy agradezco ser mamá aún difícil resulta vec...


In [10]:
import spacy

# Carga el modelo preentrenado en español
nlp = spacy.load("es_core_news_sm")

# Función para obtener el sentimiento de un texto
def obtener_sentimiento(texto):
    if isinstance(texto, str) and texto.strip():
        doc = nlp(texto)
        return doc.sentiment
    else:
        return -2

# Cargar el DataFrame original desde el archivo CSV
df_original = pd.read_csv('datasets/extras/archivo_combinado.csv')

# Agregar la columna 'sentimiento' al df_interim
df_interim['sentimiento'] = df_interim['message_clean'].apply(obtener_sentimiento)

# Combinar el DataFrame original con el df_interim usando el Identificador de la publicación
df_actualizado = df_original.merge(df_interim[['identificador_de_la_publicacion', 'sentimiento']], on='identificador_de_la_publicacion', how='left')

# Asignar -2 a las publicaciones sin captions (donde el sentimiento es NaN)
df_actualizado['sentimiento'].fillna(-2, inplace=True)





/tmp/ipykernel_65433/1906041773.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_actualizado['sentimiento'].fillna(-2, inplace=True)


In [11]:
# Eliminar las historias que son videos



# Guardar el DataFrame actualizado en un nuevo archivo CSV
df_actualizado.to_csv('datasets/extras/archivo_combinado_cleaned.csv', index=False)

# Mostrar el DataFrame con la nueva columna de sentimiento
print(df_actualizado.head())

   identificador_de_la_publicacion  identificador_de_la_cuenta  \
0                17994711058627085           17841408055500202   
1                17989247059674185           17841408055500202   
2                17953813415188610           17841408055500202   
3                17973503881980868           17841408055500202   
4                17974040296969337           17841408055500202   

  nombre_de_usuario_de_la_cuenta  \
0                    jessonirica   
1                    jessonirica   
2                    jessonirica   
3                    jessonirica   
4                    jessonirica   

                                 nombre_de_la_cuenta  \
0  Jess Onírica | Sueños, creatividad, espiritual...   
1  Jess Onírica | Sueños, creatividad, espiritual...   
2  Jess Onírica | Sueños, creatividad, espiritual...   
3  Jess Onírica | Sueños, creatividad, espiritual...   
4  Jess Onírica | Sueños, creatividad, espiritual...   

                              descripcion  duraci